In [14]:
import itertools
import os
import glob
import pandas as pd
import pickle
from conf_dict_params import config_dict_grid, traces

In [15]:
traces = [
          "./traces/WIRED_900kbps.json",
          "./traces/WIRED_35mbps.json",
          "./traces/WIRED_200kbps.json", 
          "./traces/4G_700kbps.json",
          "./traces/4G_3mbps.json",
          "./traces/4G_500kbps.json",
          "./traces/5G_12mbps.json",
          "./traces/5G_13mbps.json",
          "./traces/trace_300k.json", 
          # "./big_trace/big_trace2.json",
           ]

In [3]:
config_dict_grid = {
    "trace": traces,
    "delay_states": [False, True],
    "normalize_states": [True],
    "step_time": [200],
    "alg": ["SAC", "TD3"],
    "tuned": [False, True],
    "reward_profile": [0],
}

In [4]:
keys, values = zip(*config_dict_grid.items())
permutation_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
print("Len permutation dicts: ", len(permutation_dicts))

Len permutation dicts:  72


In [5]:
permutation_dicts

[{'trace': './traces/WIRED_900kbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': False,
  'reward_profile': 0},
 {'trace': './traces/WIRED_900kbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': True,
  'reward_profile': 0},
 {'trace': './traces/WIRED_900kbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'TD3',
  'tuned': False,
  'reward_profile': 0},
 {'trace': './traces/WIRED_900kbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'TD3',
  'tuned': True,
  'reward_profile': 0},
 {'trace': './traces/WIRED_900kbps.json',
  'delay_states': True,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': False,
  'reward_profile': 0},
 {'trace': './traces/WIRED_900kbps.json',
  'delay_states': True,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned

In [50]:
from best_algs import best_models_dict
[d[200] for d in best_models_dict.values()]

['TD3_WIRED_900kbps_200_delay_False_norm_states_True_tuned_False_reward_profile_0_seed_22',
 'TD3_WIRED_200kbps_200_delay_False_norm_states_True_tuned_True_reward_profile_0_seed_22',
 'TD3_WIRED_35mbps_200_delay_True_norm_states_True_tuned_True_reward_profile_0_seed_22',
 'SAC_4G_700kbps_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_22',
 'TD3_4G_3mbps_200_delay_True_norm_states_True_tuned_True_reward_profile_0_seed_22',
 'TD3_4G_500kbps_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_22',
 'TD3_5G_12mbps_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_22',
 'TD3_5G_13mbps_200_delay_False_norm_states_True_tuned_False_reward_profile_0_seed_22',
 'TD3_trace_300k_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_22']

## Make a custom permutation dict

In [6]:
new_permutation_dicts = []

for d in permutation_dicts:

    if (d["alg"] == "TD3"):
        new_permutation_dicts.append(d)
    
    elif (d["alg"] == "SAC") and (d["delay_states"] == True):
        new_permutation_dicts.append(d)

print(len(new_permutation_dicts))
for di in new_permutation_dicts:
    print(di)

54
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': False, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': True, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': False, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': True, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': True, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': False, 'reward_profile': 0}
{'trace': './traces/WIRED_900kbps.json', 'delay_states': True, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_35mbps.json', 'delay_sta

In [8]:
# with open('best_algs_correct_2.pkl', 'wb') as f:
#     pickle.dump(new_permutation_dicts[30:], f)

In [13]:
len(pd.read_pickle('best_algs_correct_1.pkl'))

30

## Make a custom permutaiton dict with only best alg

In [16]:
type(permutation_dicts[0]["delay_states"])

bool

In [22]:
new_permutation_dicts = []

for d in permutation_dicts:
    trace_name = d["trace"].split("/")[2].split(".")[0]
    if trace_name == 'WIRED_900kbps':
        #SAC, tuned
        if (d["alg"] == "SAC") and (d["delay_states"] == False) and (d["tuned"] == True):
            new_permutation_dicts.append(d)

    elif trace_name == "WIRED_200kbps":
        #TD3_WIRED_200kbps_200_delay_False_norm_states_True_tuned_False
        if (d["alg"] == "TD3") and (d["delay_states"] == False) and (d["tuned"] == False):
            new_permutation_dicts.append(d)

    elif trace_name == "WIRED_35mbps":
        #TD3_WIRED_35mbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["delay_states"] == False) and (d["tuned"] == True):
            new_permutation_dicts.append(d)

    elif trace_name == "4G_700kbps":
        #delay True is better but we go with delay False
        #TD3_4G_700kbps_200_delay_True_norm_states_True_tuned_False
        if (d["alg"] == "TD3") and (d["delay_states"] == True) and (d["tuned"] == True):
            new_permutation_dicts.append(d)
    
    elif trace_name == "4G_3mbps":
        #TD3_4G_3mbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["delay_states"] == False) and (d["tuned"] == True):
            new_permutation_dicts.append(d)
    
    elif trace_name == "4G_500kbps":
        #TD3_4G_500kbps_200_delay_False_norm_states_True_tuned_True
        if (d["alg"] == "TD3") and (d["delay_states"] == False) and (d["tuned"] == True):
            new_permutation_dicts.append(d)
            
    elif trace_name == "5G_12mbps":
        #TD3_5G_12mbps_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_36
        if (d["alg"] == "TD3") and (d["delay_states"] == True) and (d["tuned"] == False) and (d["reward_profile"] == 0):
            new_permutation_dicts.append(d)
            
    elif trace_name == "5G_13mbps":
        #TD3_5G_13mbps_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_36
        if (d["alg"] == "TD3") and (d["delay_states"] == True) and (d["tuned"] == False) and (d["reward_profile"] == 0):
            new_permutation_dicts.append(d)
            
    elif trace_name == "trace_300k":
        #SAC_trace_300k_200_delay_True_norm_states_True_tuned_False_reward_profile_0_seed_36
        if (d["alg"] == "SAC") and (d["delay_states"] == True) and (d["tuned"] == False) and (d["reward_profile"] == 0):
            new_permutation_dicts.append(d)

    
for di in new_permutation_dicts:
    print(di)

{'trace': './traces/WIRED_900kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'SAC', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_35mbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/WIRED_200kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': False, 'reward_profile': 0}
{'trace': './traces/4G_700kbps.json', 'delay_states': True, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/4G_3mbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/4G_500kbps.json', 'delay_states': False, 'normalize_states': True, 'step_time': 200, 'alg': 'TD3', 'tuned': True, 'reward_profile': 0}
{'trace': './traces/5G_12mbps.json', 'delay_states': True, 'norm

In [23]:
len(new_permutation_dicts)

9

In [24]:
with open('permutation_dicts_trial_qoe.pkl', 'wb') as f:
    pickle.dump(new_permutation_dicts, f)

## See what is finished and what is missing

In [66]:
data_dir_list = os.listdir("./data_mp")
output_dir_list = os.listdir("./output_mp")
tensorboard_dir_list = os.listdir("./tensorboard_logs_mp")

In [67]:
len(data_dir_list), len(output_dir_list), len(tensorboard_dir_list)

(499, 499, 346)

In [68]:
output_dir_list

['rates_delay_loss_TD3_4G_3mbps_400_delay_False_norm_states_True_tuned_True.pickle',
 'rates_delay_loss_SAC_big_trace2_600_delay_False_norm_states_True_tuned_False.pickle',
 'rates_delay_loss_SAC_WIRED_200kbps_400_delay_False_norm_states_True_tuned_False.pickle',
 'rates_delay_loss_SAC_WIRED_200kbps_400_delay_True_norm_states_False_tuned_True.pickle',
 'rates_delay_loss_SAC_WIRED_900kbps_400_delay_False_norm_states_True_tuned_True.pickle',
 'rates_delay_loss_TD3_4G_700kbps_400_delay_False_norm_states_True_tuned_True.pickle',
 'rates_delay_loss_SAC_trace_300k_200_delay_False_norm_states_True_tuned_True_reward_profile_0_seed_36.pickle',
 'rates_delay_loss_PPO_WIRED_35mbps_600_delay_True_norm_states_False_tuned_True.pickle',
 'rates_delay_loss_TD3_WIRED_35mbps_200_delay_False_norm_states_True_tuned_True_reward_profile_2_seed_36.pickle',
 'rates_delay_loss_TD3_WIRED_200kbps_200_delay_False_norm_states_True_tuned_False_reward_profile_3_seed_36.pickle',
 'rates_delay_loss_SAC_5G_12mbps_200_d

In [69]:
permutations_done = []
permutations_there_but_not_done = []

for d in output_dir_list:
    
    if (not "5G" in d) and (not "trace_300k" in d):
        continue
    
    pickle_path = "output_mp/" + d
    l_rate = pd.read_pickle(pickle_path)
    trace = list(l_rate.keys())[0]
    if len(l_rate[trace]) < 10:
        print(len(l_rate[trace]))
        permutations_there_but_not_done.append(d)
    else: 
        d_splitted = d.split("_")
        alg = d_splitted[3]
        trace_name = d_splitted[4] + "_" + d_splitted[5]
        trace = "./traces/" + trace_name + ".json"
        step_time = int(d_splitted[6])
        
        reward_profile = int(d_splitted[16])

        if d_splitted[8] == "True":
            delay_states = True
        else:
            delay_states = False
        if d_splitted[11] == "True":
            normalize_states = True
        else:
            normalize_states = False
        if d_splitted[13].split(".")[0] == "True":
            tuned = True
        else:
            tuned = False

        d_final = {
         'trace': trace,
         'delay_states': delay_states,
         'normalize_states': normalize_states,
         'step_time': step_time,
         'alg': alg,
         'tuned': tuned,
         'reward_profile': reward_profile,
        }

        permutations_done.append(d_final)

In [70]:
permutations_there_but_not_done

[]

In [53]:
permutations_there_but_not_done_dicts = []

for d in permutations_there_but_not_done:
    d_splitted = d.split("_")
    alg = d_splitted[3]
    trace_name = d_splitted[4] + "_" + d_splitted[5]
    trace = "./traces/" + trace_name + ".json"
    step_time = int(d_splitted[6])

    reward_profile = int(d_splitted[16])

    if d_splitted[8] == "True":
        delay_states = True
    else:
        delay_states = False
    if d_splitted[11] == "True":
        normalize_states = True
    else:
        normalize_states = False
    if d_splitted[13].split(".")[0] == "True":
        tuned = True
    else:
        tuned = False

    d_final = {
     'trace': trace,
     'delay_states': delay_states,
     'normalize_states': normalize_states,
     'step_time': step_time,
     'alg': alg,
     'tuned': tuned,
     'reward_profile': reward_profile,
    }
    
    permutations_there_but_not_done_dicts.append(d_final)

In [54]:
permutations_there_but_not_done_dicts

[{'trace': './traces/5G_13mbps.json',
  'delay_states': True,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': False,
  'reward_profile': 3},
 {'trace': './traces/5G_13mbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': True,
  'reward_profile': 2},
 {'trace': './traces/5G_13mbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': True,
  'reward_profile': 4},
 {'trace': './traces/5G_13mbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'TD3',
  'tuned': True,
  'reward_profile': 4},
 {'trace': './traces/5G_13mbps.json',
  'delay_states': True,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': False,
  'reward_profile': 2},
 {'trace': './traces/5G_13mbps.json',
  'delay_states': False,
  'normalize_states': True,
  'step_time': 200,
  'alg': 'SAC',
  'tuned': True,
  'reward_profil

In [71]:
len(permutations_done)

120

In [72]:
len(permutation_dicts) - len(permutations_done)

0

In [73]:
permutations_to_do = [i for i in permutation_dicts if i not in permutations_done]
len(permutations_to_do)

0

In [60]:
p_final = permutations_to_do + permutations_there_but_not_done_dicts
type(p_final[-1]["reward_profile"])

int

In [61]:
with open('5G_permutations_last.pkl', 'wb') as f:
    pickle.dump(p_final, f)

In [ ]:
real_permutations_to_do = []

for p in permutations_to_do:
    if (p["normalize_states"] == True) and (p["alg"] != "PPO") and (p["trace"] != './big_trace/big_trace2.json'):
        real_permutations_to_do.append(p)

In [ ]:
len(real_permutations_to_do)

In [ ]:
real_permutations_to_do

In [ ]:
with open('permutation_dicts_big_trace_only.pkl', 'wb') as f:
    pickle.dump(real_permutations_to_do, f)

In [ ]:
len(pd.read_pickle('permutation_dicts_big_trace_only.pkl'))

In [ ]:
for i in range(3,6):
    print(i)

## Deleting permutations_there_but_not_done - uncommment

In [ ]:
for fname in permutations_there_but_not_done:
    fname = "./output_mp/" + fname
    print(fname)
    if os.path.isfile(fname): # this makes the code more robust
        # os.remove(fname)
        continue

In [ ]:
# to_delete = [
# "SAC_WIRED_900kbps_400_delay_False_norm_states_True_tuned_True",
# "SAC_WIRED_900kbps_600_delay_True_norm_states_False_tuned_True",
# "PPO_WIRED_900kbps_400_delay_True_norm_states_True_tuned_False",
# "PPO_WIRED_900kbps_400_delay_False_norm_states_False_tuned_True",
# "TD3_WIRED_35mbps_200_delay_True_norm_states_True_tuned_False"  
# ]